# Operations Rehearsal for Commissioning #4 June 2024 LSSTComCamSim Data Access Demo

Tutorial notebook for accessing data from the Operations Rehearsal for Commissioning #4 in June 2024 with pointers to other resources, examples, and documentation. Most examples use simulated LSSTComCam data stream.

More detailed examples of access patterns can be found in the `ops_rehearsal_comcam_analysis.ipynb` notebook.

Last verified to run: 26 June 2024

LSST Science Pipelines Version: `w_2024_25`

## What version of the Science Pipelines am I using?

In [ ]:
!eups list -s | grep lsst_distrib

For a summary of changes between versions, see https://lsst-dm.github.io/lsst_git_changelog/

Import additional python packages that we will need for this tutorial.

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

## How do I find information on data processing campaigns?

See https://confluence.lsstcorp.org/display/DM/Campaigns for the list of productions that the campaign management team has acknowledged as routine and accepted to oversee.

In particular, there are multiple campaigns for Operations Rehearsal #4 using the simulated LSSTComCam data stream.

The repo and collection names are provided, which we will need for the next section.

## How do I instatiate a Butler for data access?

* https://github.com/rubin-dp0/tutorial-notebooks/blob/main/DP02_01_Introduction_to_DP02.ipynb

In [ ]:
from lsst.daf.butler import Butler

In [ ]:
# One can see all of the collections with the following command
repo = 'embargo_or4'
butler = Butler(repo)
# Example search for Nightly Validation pipeline collections
for _ in butler.registry.queryCollections():
    if 'LSSTComCamSim/runs/nightlyValidation/' in _:
        print(_)

In [ ]:
repo = 'embargo_or4'
# Intranight
#collection = 'LSSTComCamSim/nightlyValidation'
# 10am processing
# collection = 'LSSTComCamSim/runs/nightlyValidation/{day_obs}/<lsst_distrib_tag>/DM-44966'
# Examples of specific nights
# collection = 'LSSTComCamSim/runs/nightlyValidation/20240625/w_2024_25/DM-44966'
collection = 'LSSTComCamSim/runs/nightlyValidation/20240626/w_2024_25/DM-44966'

butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
# Determine which dataset types exist in the collection
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(execute=False, exact=False):
        # Limit search results to the data products
        if ('_config' not in datasetType.name) and ('_log' not in datasetType.name) and ('_metadata' not in datasetType.name) and ('_resource_usage' not in datasetType.name):
            print(datasetType)

## Standard Data Model

We will be accessing several tabular data products. A quick reference for the Standard Data Model schema:

https://dm.lsst.org/sdm_schemas/browser/

In [ ]:
objectTable_refs = sorted(butler.registry.queryDatasets('objectTable_tract'))
objectTable = butler.get(objectTable_refs[0])
objectTable